# Selective flow constraints with [PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows and Sourabh Dalvi

## Introduction

The [Operations Problems example]](../../notebook/3_PowerSimulations_examples/sequential_simulations.ipynb)
shows the basic building blocks of building optimization problems with PowerSimulations.jl.
This example shows how to customize the enforcement of branch flow constraints as is common
when trying to build large scale simulations.

## Dependencies

In [1]:
using SIIPExamples

### Modeling Packages

In [2]:
using PowerSystems
using PowerSimulations

### Optimization packages
For this simple example, we can use the Cbc solver with a relatively relaxed tolerance.

In [3]:
using Cbc #solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.5])

### Create a `System` from RTS-GMLC data
We can just use the
[parsing tabular data example.](../../notebook/2_PowerSystems_examples/parse_tabulardata.jl)

In [4]:
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath, "test", "2_PowerSystems_examples", "parse_tabulardata.jl"))

┌ Info: Parsing csv files in Reserves ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:125
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Down.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:129
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Up.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:129
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Down.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:129
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Up.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:129
┌ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R1.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/pow

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,109
2,Area,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
3,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,73
4,GenericBattery,Storage <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
5,HVDCLine,DCBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
6,HydroDispatch,HydroGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
7,HydroEnergyReservoir,HydroGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,19
8,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,105
9,LoadZone,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,21


### Selecting flow limited lines
Since PowerSimulations will apply constraints by component type (e.g. Line), we need to
change the component type of the lines on which we want to enforce flow limits. So, let's
change the device type of certain branches from Line to MonitoredLine differentiate
treatment when we build the model. Here, we can select inter-regional lines, or lines
above a voltage threshold.

In [5]:
for line in get_components(Line, sys)
    if (get_base_voltage(get_from(get_arc(line))) >= 230.0) &&
       (get_base_voltage(get_to(get_arc(line))) >= 230.0)
        #if get_area(get_from(get_arc(line))) != get_area(get_to(get_arc(line)))
        @info "Changing $(get_name(line)) to MonitoredLine"
        convert_component!(MonitoredLine, line, sys)
    end
end

┌ Info: Changing C31-2 to MonitoredLine
└ @ Main.##267 string:5
┌ Info: Changing B26 to MonitoredLine
└ @ Main.##267 string:5
┌ Info: Changing A32-2 to MonitoredLine
└ @ Main.##267 string:5
┌ Info: Changing CA-1 to MonitoredLine
└ @ Main.##267 string:5
┌ Info: Changing B34 to MonitoredLine
└ @ Main.##267 string:5
┌ Info: Changing B19 to MonitoredLine
└ @ Main.##267 string:5
┌ Info: Changing B31-2 to MonitoredLine
└ @ Main.##267 string:5
┌ Info: Changing C30 to MonitoredLine
└ @ Main.##267 string:5
┌ Info: Changing B27 to MonitoredLine
└ @ Main.##267 string:5
┌ Info: Changing A21 to MonitoredLine
└ @ Main.##267 string:5
┌ Info: Changing A32-1 to MonitoredLine
└ @ Main.##267 string:5
┌ Info: Changing B29 to MonitoredLine
└ @ Main.##267 string:5
┌ Info: Changing A18 to MonitoredLine
└ @ Main.##267 string:5
┌ Info: Changing C24 to MonitoredLine
└ @ Main.##267 string:5
┌ Info: Changing C28 to MonitoredLine
└ @ Main.##267 string:5
┌ Info: Changing A29 to MonitoredLine
└ @ Main.##267 string:5

## Build an `OperationsProblem`

In [6]:
uc_prob =
    UnitCommitmentProblem(sys, optimizer = solver, horizon = 24, network = DCPPowerModel)

┌ Info: Unit System changed to SYSTEM_BASE
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/base.jl:282
┌ Warning: The data doesn't include devices of type InterruptibleLoad, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/0nHyl/src/devices_models/device_constructors/common/constructor_validations.jl:3
┌ Warning: No variable cost defined for 201_HYDRO_4
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/0nHyl/src/devices_models/devices/common/cost_functions.jl:337
┌ Warning: The data doesn't include devices of type Transformer2W, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/0nHyl/src/devices_models/device_constructors/common/constructor_validations.jl:3



Operations Problem Specification

  transmission:  PowerModels.DCPPowerModel
  devices: 
      ILoads:
        device_type = InterruptibleLoad
        formulation = PowerSimulations.InterruptiblePowerLoad
      HydroROR:
        device_type = HydroDispatch
        formulation = PowerSimulations.HydroDispatchRunOfRiver
      Generators:
        device_type = ThermalStandard
        formulation = PowerSimulations.ThermalBasicUnitCommitment
      DistRE:
        device_type = RenewableFix
        formulation = PowerSimulations.FixedOutput
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = PowerSimulations.HydroDispatchRunOfRiver
      Loads:
        device_type = PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      RE:
        device_type = RenewableDispatch
        formulation = PowerSimulations.RenewableFullDispatch
  branches: 
      T:
        device_type = Transformer2W
        formulation = PowerSimulations.StaticTransformer
      TT:
  

Let's change the formulation of the `Line` components to an unbounded flow formulation.
This formulation still enforces Kirchoff's laws, but does not apply flow constraints.

In [7]:
set_branch_model!(uc_prob, :L, DeviceModel(Line, StaticLineUnbounded))

┌ Info: Unit System changed to SYSTEM_BASE
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/base.jl:282
┌ Warning: The data doesn't include devices of type InterruptibleLoad, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/0nHyl/src/devices_models/device_constructors/common/constructor_validations.jl:3
┌ Warning: No variable cost defined for 201_HYDRO_4
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/0nHyl/src/devices_models/devices/common/cost_functions.jl:337
┌ Warning: The data doesn't include devices of type Transformer2W, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/0nHyl/src/devices_models/device_constructors/common/constructor_validations.jl:3


Notice that there is no entry for `MonitoredLine` branches. So, let's add one.

In [8]:
construct_device!(uc_prob, :ML, DeviceModel(MonitoredLine, StaticLine))

Solve the relaxed problem

In [9]:
solve!(uc_prob)

Results
========

Variables
=========

P__ThermalStandard: (24, 76)
P__RenewableDispatch: (24, 30)
Reg_Down__VariableReserve_ReserveDown: (24, 102)
P__HydroEnergyReservoir: (24, 19)
Flex_Down__VariableReserve_ReserveDown: (24, 102)
Fp__MonitoredLine: (24, 68)
Reg_Up__VariableReserve_ReserveUp: (24, 102)
P__HydroDispatch: (24, 1)
Spin_Up_R2__VariableReserve_ReserveUp: (24, 25)
theta__Bus: (24, 73)
On__ThermalStandard: (24, 76)
Fp__Line: (24, 37)
start__ThermalStandard: (24, 76)
Flex_Up__VariableReserve_ReserveUp: (24, 102)
Fp__TapTransformer: (24, 15)
Fp__HVDCLine: (24, 1)
stop__ThermalStandard: (24, 76)
Spin_Up_R1__VariableReserve_ReserveUp: (24, 34)
Spin_Up_R3__VariableReserve_ReserveUp: (24, 43)


Optimizer Log
-------------
        timed_solve_time = 5.889575318
        solve_bytes_alloc = 113404512
        solve_time = 5.3492138385772705
        obj_value = 1.0876769066849346e6
        solver = COIN Branch-and-Cut (Cbc)
        sec_in_gc = 0.079311351
        dual_status = NO_SOLUTION
        primal_status = FEASIBLE_POINT
        termination_status = OPTIMAL


Total Cost: OBJECTIVE_FUNCTION = 1.0876769066849346e6

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*